# Initializations

In [ ]:
import gym
import torch
import matplotlib.pyplot as plt
import numpy as np
from copy import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from enum import Enum
from collections import deque
from random import random
from copy import deepcopy
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from operator import add
from IPython.display import display, clear_output

# Environment exploration

## Cartpole-v1

### sample components

In [ ]:
cartpole_env = gym.make('CartPole-v1')
state = cartpole_env.reset()
print("\n***action***")
print(cartpole_env.action_space)
print("\n***State***")
print(state, cartpole_env.state.shape)
print("observationspace ", cartpole_env.observation_space.shape[0])
print("actionspace," ,cartpole_env.action_space.n)


state, reward, done, _ = cartpole_env.step(cartpole_env.action_space.sample())

print("\n*** reward ***")
print(reward)

### visualizing env

In [ ]:
cartpole_env.reset()
for _ in range(1000):
    cartpole_env.render()
    _, _, done, _ = cartpole_env.step(cartpole_env.action_space.sample())
    if done:
        break
cartpole_env.close()

## LunarLander-v2

### sample components

In [ ]:
lun_env = gym.make('LunarLander-v2')
print("\n*** action ***")
print(lun_env.action_space)

state = lun_env.reset()
print("\n*** state ***")
print(state, state.shape)

s, r, done, _ = lun_env.step(lun_env.action_space.sample())

print("\n*** reward ***")
print(r)

### visualizing env

In [ ]:
lun_env.reset()
for _ in range(1000):
    lun_env.render()
    _, _, done, _ = lun_env.step(lun_env.action_space.sample())
    if done:
        break
lun_env.close()

# DQN

## Define q-value architecture

In [ ]:

class FCNet(nn.Module):

    def __init__(self, input_size, output_size, device):
        super(FCNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 300)
        self.fc2 = nn.Linear(300, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

## Define agent

In [ ]:

class DqnVannila:
    def __init__(self, sync_steps, buffer_size, gamma, lr, eps, state_size, action_size, batch_size, device):
        self.sync_steps = sync_steps
        self.max_buffer_size = buffer_size
        self.gamma = torch.tensor(gamma)
        self.lr = lr
        self.eps = eps
        self.buffer = deque()
        self.action_size = action_size
        self.q = FCNet(state_size, action_size, device=device)
        self.q_target = FCNet(state_size, action_size, device=device)
        self.batch_size = batch_size
        self.device = device
        self.optimizer = optim.Adam(self.q.parameters(), lr=lr)
        self.update_step = 0
        
    def get_q_val(self, qnet, state, action=None):
        if action is None:
            return qnet(state)
        return qnet(state).gather(1, action.reshape(-1,1).long())
        
    def add_to_buffer(self, experience):
        if len(self.buffer) >= self.max_buffer_size:
            self.buffer.pop()
        self.buffer.appendleft(experience)
        
    def _np_to_tensor(self, nparrs):
        return [torch.from_numpy(arr).float().to(self.device) for arr in nparrs]
        
    def sample_minibatch(self):
        idxs = np.random.randint(0, len(self.buffer), self.batch_size)
        sample = [self.buffer[i] for i in idxs]
        
        states, actions, rewards, next_states, dones = list(map(lambda x: np.array(x, dtype=np.float64), zip(*sample)))
        return tuple(self._np_to_tensor([states, actions, rewards, next_states, dones]))
    
    def synchronize(self):
        self.q_target.load_state_dict(self.q.state_dict())
        
    def get_action(self, state):
        state = torch.from_numpy(state).float().to(self.device)
        if random() < self.eps:
            return np.random.randint(0, self.action_size)
        else:
            with torch.no_grad():
                return np.argmax(self.get_q_val(self.q, state).numpy())
        
    def update(self, state, action, reward, next_state, done):
        self.update_step += 1
        
        #TODO:implement epsilon decay properly
        new_eps = self.eps - (self.update_step/1000)*self.eps
        self.eps = max(0, new_eps)
        
        self.add_to_buffer([state, action, reward, next_state, done])
        
        #TODO: warmup. should be configurable.
        if len(self.buffer) < 1000:
            return
        
        batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones = self.sample_minibatch()
        
        with torch.no_grad():
            last_step_target = batch_dones * batch_rewards
            middle_step_target = (1-batch_dones)*(batch_rewards + self.gamma*torch.max(self.get_q_val(self.q_target, batch_next_states), dim=1).values)
            targets =  last_step_target + middle_step_target
        
        predictions = self.get_q_val(self.q, batch_states, batch_actions).squeeze()
        loss = F.mse_loss(predictions, targets)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        if self.update_step % self.sync_steps == 0:
            self.synchronize()
        


## nasty training for test (should be removed later)

In [ ]:
env = gym.make('CartPole-v1')
done = False
state = env.reset()

dqn = DqnVannila(
    sync_steps=100,
    buffer_size=1000,
    gamma=0.99,
    lr=0.001,
    eps=0.6,
    state_size=env.observation_space.shape[0],
    action_size=env.action_space.n,
    batch_size=128,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
)

#TODO: implement a trainer function which does the following:
#      takes epochs, env and solver -> trains and returns required graphs and metrics
returns = []
for _ in tqdm(range(1000)):
    ret = 0
    while not done:
        a = dqn.get_action(state)
        next_state, r, done, _ = env.step(a)
        dqn.update(state, a, r, next_state, done)
        ret += r
        state = next_state
    done = False
    env.reset()
    returns.append(ret)
    
plt.plot(returns)

# GridWorld environment

## Defining environment objects

In [ ]:
class WorldObj:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return " "
    
class Empty(WorldObj):
    def __init__(self):
        super().__init__('empty')
    
class Log(WorldObj):
    def __init__(self):
        super().__init__('log')
    
    def __repr__(self):
        return '_'
    
class Gold(WorldObj):
    def __init__(self):
        super().__init__('gold')
    
    def __repr__(self):
        return 'G'
    
class Food(WorldObj):
    def __init__(self):
        super().__init__('food')
    
    def __repr__(self):
        return '8'
    
    
class Trap(WorldObj):
    def __init__(self):
        super().__init__('trap')
        self.state = None
    
    def __repr__(self):
        return 'X'

class Teleport(WorldObj):
    def __init__(self, dest, chance=0.8):
        super().__init__('teleport')
        self.destination = dest
        self.chance = chance
    
    def __repr__(self):
        return 'T'
    
    
class Stream(WorldObj):
    """
    stream only goes down or right!
    """
    
    def __init__(self, _dir: int):
        super().__init__('stream')
        self.dir = _dir
    
    def __repr__(self):
        return '~'
    
class Miner(WorldObj):
    def __init__(self):
        super().__init__('miner')
    
    def __repr__(self):
        return '*'
        

## Defining environment

In [ ]:
class Action(Enum):
    RIGHT = 0
    DOWN = 1
    LEFT = 2
    UP = 3
    LOG_RIGHT = 4
    LOG_DOWN = 5
    LOG_LEFT = 6
    LOG_UP = 7
    
    

class GoldHuntEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}
    static_map_size = 7
    movement_acts = [Action.RIGHT, Action.LEFT, Action.DOWN, Action.UP,]
    craft_acts = [Action.LOG_RIGHT, Action.LOG_LEFT, Action.LOG_DOWN, Action.LOG_UP,]

    def __init__(self, ascii_rep=True, food_prob=0.6, teleport_prob=0.8):
        self.ascii_rep = ascii_rep
        self.action_space = gym.spaces.Discrete(8)
        self.observation_space = gym.spaces.Box(-1, 1, shape=(2,))
        self.info = {}
        self.static_map_size = 7
        self.max_time_step = self.static_map_size * 4
        self.max_reward_size = 10
        self.food_reward = 5 / self.max_reward_size
        self.gold_reward = 10 / self.max_reward_size
        self.time_punishment = -1 / self.max_reward_size
        self.trap_falling_reward = -5 / self.max_reward_size
        self.craft_log_reward = -3 / self.max_reward_size
        self.gold_loc = [6, 6]
        self.food_prob = food_prob
        self.teleport_prob = teleport_prob
        print("observation_Space", self.observation_space.shape)
        print("action_space ", self.action_space.n)
        
    def generate_static_map(self, agent_pos):
        env_map = [[Empty() for _ in range(self.map_size)] for _ in range(self.map_size)]
        
        env_map[2][0] = Trap()
        env_map[2][1] = Trap()
        env_map[2][2] = Trap()
        env_map[2][3] = Trap()
        env_map[1][3] = Trap()
        env_map[0][3] = Trap()
        env_map[5][3] = Trap()
        env_map[6][3] = Trap()
        
        env_map[self.gold_loc[0]][self.gold_loc[1]] = Gold()
        
        if self.teleport_prob > 0:
            env_map[0][6] = Teleport(dest=[6,0])
            
        env_map[4][0] = Stream(_dir=Action.LEFT.value)
        env_map[4][1] = Stream(_dir=Action.LEFT.value)
        env_map[4][2] = Stream(_dir=Action.LEFT.value)
        env_map[4][3] = Stream(_dir=Action.LEFT.value)
        env_map[4][4] = Stream(_dir=Action.LEFT.value)
        env_map[4][5] = Stream(_dir=Action.LEFT.value)
        env_map[4][6] = Stream(_dir=Action.LEFT.value)
        
            
        return env_map

    def get_pos_obj(self, pos):
        return self.env_map[pos[0]][pos[1]]

    def stochastic_generation(self, pos, obj, prob):
        if random() < prob:
            self.env_map[pos[0]][pos[1]] = obj
    
    def _get_next_pos(self, current_loc, action):
        action_dir = Action(action)
        next_pos = copy(current_loc)
        
        if action_dir not in self.movement_acts:
            return current_loc
        
        if action_dir == Action.RIGHT:
            next_pos[1] = self._restricted_move(next_pos[1], 1)
        elif action_dir == Action.LEFT:
            next_pos[1] = self._restricted_move(next_pos[1], -1)
        elif action_dir == Action.UP:
            next_pos[0] = self._restricted_move(next_pos[0], -1)
        elif action_dir == Action.DOWN:
            next_pos[0] = self._restricted_move(next_pos[0], 1)
            
        return next_pos
        
        
    def _restricted_move(self, init_val, res):
        new_val = init_val + res
        if new_val >= self.map_size:
            new_val = self.map_size - 1
        elif new_val < 0:
            new_val = 0
        return new_val
    
    
    def pos_to_state(self, pos):
        mapped_pos = [0, 0]
        mapped_pos[0] = pos[0] / (self.map_size/2) - 1
        mapped_pos[1] = pos[1] / (self.map_size/2) - 1
        return np.array(mapped_pos)
    
    def on_border(self, pos):
        if pos[0] == 0 or pos[0] == self.map_size-1 or pos[1] == 0 or pos[1] == self.map_size-1:
            return True
        return False
    
    
    def craft_log(self, current_pos, action: int):
        """
        returns True if log is crafted, False otherwise
        """
        log_loc = self._get_next_pos(current_pos, action-4)
        if log_loc == current_pos or log_loc == self.gold_loc:
            return False
        
        self.agent_pos = log_loc
        self.env_map[log_loc[0]][log_loc[1]] = Log()
        return True
    
    def generate_stochastic_elements(self):
        self.stochastic_generation([1,6], Food(), self.food_prob)
        self.stochastic_generation([0,5], Food(), self.food_prob)
        self.stochastic_generation([1,5], Food(), self.food_prob)
        self.stochastic_generation([5,1], Food(), self.food_prob)
        self.stochastic_generation([5,0], Food(), self.food_prob)
        self.stochastic_generation([6,1], Food(), self.food_prob)
    
    def step(self, action: int):
        self.info['statu'] = ""
        self.info['step'] = self.time_step
        self.time_step += 1
        
        done = self.time_step > self.max_time_step 
        reward = self.time_punishment
        
        self.info['action'] = f"agent performed {Action(action)}"
        curr_obj = self.get_pos_obj(self.agent_pos)
        
        # agent was trapped before
        if isinstance(curr_obj, Trap) and Action(action) in self.movement_acts:
            # this part shows that trap needs memory 
            #             trap_state = curr_obj.state
#             if trap_state is None or action != (trap_state+2)%4:
#                 self.info['status'] = f"Agent in trap with previous action of {None if curr_obj.state is None else Action(curr_obj.state)}"
#                 curr_obj.state = action
#                 self.info['cum_r'] += reward
#                 return self.pos_to_state(self.agent_pos), reward, done, self.info
#             else:
#                 self.info['status'] = f"Agent broke the trap"
#                 self.env_map[self.agent_pos[0]][self.agent_pos[1]] = Empty()
            self.env_map[self.agent_pos[0]][self.agent_pos[1]] = Empty()        
        
        # agent crafts a log
        if Action(action) in self.craft_acts:
            is_crafted = self.craft_log(self.agent_pos, action)
            
            if is_crafted:
                self.info['status'] = 'agent crafted a log'
                reward = self.craft_log_reward
            
            self.info['cum_r'] += reward
            return self.pos_to_state(self.agent_pos), reward, done, self.info
        

        next_pos = self._get_next_pos(self.agent_pos, action)
        next_obj = self.get_pos_obj(next_pos)

        # stream takes agent when he falls into it
        if isinstance(next_obj, Stream):
            self.info['status'] = ('agent fell into a stream')
            while isinstance(next_obj, Stream):
                next_pos = self._get_next_pos(next_pos, next_obj.dir)
                next_obj = self.get_pos_obj(next_pos)
                
                if isinstance(next_obj, Stream) and self.on_border(next_pos):
                    if next_pos[0] == self.map_size-1 and next_pos[1] == self.map_size-1:
                        done = True
                        reward = -20
                    elif next_pos[0] == self.map_size-1 and Action(next_obj.dir) == Action.DOWN:
                        next_pos[1] = self._restricted_move(next_pos[1], 1)
                    elif next_pos[0] == 0 and Action(next_obj.dir) == Action.UP:
                        next_pos[1] = self._restricted_move(next_pos[1], -1)
                    elif next_pos[1] == self.map_size-1 and Action(next_obj.dir) == Action.RIGHT:
                        next_pos[0] = self._restricted_move(next_pos[0], 1)
                    elif next_pos[1] == 0 and Action(next_obj.dir) == Action.LEFT:
                        next_pos[0] = self._restricted_move(next_pos[0], -1)
                    
                next_obj = self.get_pos_obj(next_pos)                              
        
        # gets hurt by falling into trap
        if isinstance(next_obj, Trap):
            self.info['status'] = 'agent fell into a trap'
            reward = self.trap_falling_reward
            
        if isinstance(next_obj, Teleport):
            """ manually non deterministic setup for this env """
            
            if random() <= next_obj.chance:
                self.info['status'] = f'agent teleported to {next_obj.destination} '
                self.agent_pos = next_obj.destination
                self._clear_pos(next_obj.destination)
                return self.pos_to_state(self.agent_pos), reward, done, self.info
        
        # refills energy by eating food
        if isinstance(next_obj, Food):
            self.info['status'] = 'agent ate food!'
            self._clear_pos(next_pos)
            reward = self.food_reward
        
        # agent finds the gold
        if isinstance(next_obj, Gold):
            self.info['status'] = 'agent found the GOLD!'
            done = True
            reward = self.gold_reward
            
        self.agent_pos = next_pos
        self.info['cum_r'] += reward
        
        return self.pos_to_state(self.agent_pos), reward, done, self.info

    def _clear_pos(self, pos):
        self.env_map[pos[0]][pos[1]] = Empty()
    
    def reset(self):
        self.info = {
            'cum_r': 0,
            'action': "",
            'status': "",
            'step': 0
        }
        
        self.map_size = self.static_map_size
        self.agent_pos = [0, 0]
        self.env_map = self.generate_static_map(self.agent_pos)    
        self.generate_stochastic_elements()
        self.prev_action = None
        self.time_step = 0
        
        return self.pos_to_state(self.agent_pos)
  
    def get_env_rep(self):
        env_map_top_layer = deepcopy(self.env_map)
        env_map_top_layer[self.agent_pos[0]][self.agent_pos[1]] = Miner()
        
        return tabulate(env_map_top_layer, headers=[], tablefmt='grid')

    def get_rgb_rep(self):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, self.map_size)
        ax.set_ylim(0, self.map_size)
        
        show_layer = deepcopy(self.env_map)
        show_layer[self.agent_pos[0]][self.agent_pos[1]] = Miner()
        
        for i in range(len(show_layer)):
            for j in range(len(show_layer)):
                obj = show_layer[i][j]
                
                if isinstance(obj, Gold):
                    path = './images/gold.png'
                elif isinstance(obj, Trap):
                    path = './images/trap.png'
                elif isinstance(obj, Food):
                    path = './images/food.png'
                elif isinstance(obj, Log):
                    path = './images/log.png'
                elif isinstance(obj, Stream):
                    path = './images/stream.png'
                elif isinstance(obj, Teleport):
                    path = './images/teleport.png'
                elif isinstance(obj, Miner):
                    path = './images/agent.png'
                else:
                    continue
                    
                
                obj_repr = AnnotationBbox(OffsetImage(plt.imread(path), zoom=0.28),
                       list(map(add, [i, j], [0.5, 0.5])), frameon=False)
                ax.add_artist(obj_repr)
        
        plt.xticks([i for i in range(self.map_size)])
        plt.yticks([i for i in range(self.map_size)])
        plt.grid()
        plt.show()
        
    
    def render(self, mode='human'):
        if self.ascii_rep:
            print(self.get_env_rep())
        else:
            self.get_rgb_rep()
        

    def close(self):
        pass
    
env = GoldHuntEnv(ascii_rep=False)
env.reset()
env.render()

## training dqn on gridworld

In [ ]:
def trainDQN(env, buffer_size, gamma_val, learning_rate, epsilon_val, batch_size):
    done = False
    state = env.reset()
    
    dqn = DqnVannila(
        sync_steps=100,
        buffer_size=buffer_size,
        gamma=gamma_val,
        lr=learning_rate,
        eps=epsilon_val,
        state_size=env.observation_space.shape[0],
        action_size=(env.action_space.n),
        batch_size=128,
        device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    )

    returns = []
    for _ in tqdm(range(1000)):
        ret = 0
        while not done:
            a = dqn.get_action(state)
            next_state, r, done, _ = env.step(a)
            dqn.update(state, a, r, next_state, done)
            ret += r
            state = next_state
        done = False
        env.reset()
        returns.append(ret)

    plt.plot(returns)
    return dqn
    
gold_hunt_env = GoldHuntEnv(ascii_rep=False)
gold_hunt_env.reset()

dqn = trainDQN(
    env = gold_hunt_env,
    buffer_size=1000,
    gamma_val=0.99,
    learning_rate=0.01,
    epsilon_val=0.06,
    batch_size=128
)

## play in environment

In [ ]:
import time

def play(env, agent):
    done = False
    clear_output(wait=True)
    state = env.reset()
    env.render()
    
    while not done:
        action = agent.get_action(state)
        state, reward, done, info = env.step(action)
        
        clear_output(wait=True)
        print(info)
        env.render()
        time.sleep(0.5)
        
        
play(gold_hunt_env, dqn)